<a href="https://colab.research.google.com/github/mralamdari/Computer-Vision-Papers/blob/main/FFSSD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import torch
import torchvision
from IPython.display import clear_output

!pip install torch_intermediate_layer_getter
from torch_intermediate_layer_getter import IntermediateLayerGetter

tf.__version__

'2.12.0'

#tf

In [3]:
# vgg16 = tf.keras.applications.vgg16.VGG16(weights='imagenet')

In [4]:
# vgg16.summary()

In [5]:
# for i in range(len(vgg16.layers)):
#   vgg16.layers[i].trainable=False
#   print(vgg16.layers[i].name)

In [6]:
def ftb(current_layer, former_layer=None):
  x = tf.keras.layers.Conv2D(filters=256, kernel_size=3, strides=1, activation='relu', padding='same')(current_layer)
  x = tf.keras.layers.Conv2D(filters=256, kernel_size=3, padding='same')(x)

  if former_layer is None:
    return x
  
  elif x.shape == former_layer.shape:
    return tf.keras.layers.Add()([x, former_layer])

  else:
    d = tf.keras.layers.Conv2DTranspose(filters=256, kernel_size=4, strides=2, padding='same')(former_layer)
    return tf.keras.layers.Add()([x, d])

In [7]:
def rfem(x):
  channel = x.shape[-1]
  #branch1
  
  x1 = tf.keras.layers.MaxPool2D(pool_size=2, padding='same', strides=1)(x)
  x1 = tf.keras.layers.Conv2D(filters=channel//4, kernel_size=(1, 1), padding='same', activation='relu')(x1)
  x1 = tf.keras.layers.BatchNormalization()(x1)

  #branch2
  x2 = tf.keras.layers.Conv2D(filters=channel//4, kernel_size=(1, 1), padding='same', activation='relu')(x)
  x2 = tf.keras.layers.BatchNormalization()(x2)
  x2 = tf.keras.layers.Conv2D(filters=channel//4, kernel_size=(1, 3), padding='same', activation='relu')(x2)
  x2 = tf.keras.layers.BatchNormalization()(x2)

  #branch3
  x3 = tf.keras.layers.Conv2D(filters=channel//4, kernel_size=(1, 1), padding='same', activation='relu')(x)
  x3 = tf.keras.layers.BatchNormalization()(x3)
  x3 = tf.keras.layers.Conv2D(filters=channel//4, kernel_size=(3, 1), padding='same', activation='relu')(x3)
  x3 = tf.keras.layers.BatchNormalization()(x3)

  #branch4
  x4 = tf.keras.layers.Conv2D(filters=channel//4, kernel_size=(1, 1), padding='same', activation='relu')(x)
  x4 = tf.keras.layers.BatchNormalization()(x4)
  x4 = tf.keras.layers.Conv2D(filters=channel//3, kernel_size=(3, 1), padding='same', activation='relu')(x4)
  x4 = tf.keras.layers.BatchNormalization()(x4)
  x4 = tf.keras.layers.Conv2D(filters=channel//3, kernel_size=(1, 3), padding='same', activation='relu')(x4)
  x4 = tf.keras.layers.BatchNormalization()(x4)
  x4 = tf.keras.layers.Conv2D(filters=channel//4, kernel_size=(3, 1), padding='same', activation='relu')(x4)
  x4 = tf.keras.layers.BatchNormalization()(x4)
  x4 = tf.keras.layers.Conv2D(filters=channel//4, kernel_size=(1, 3), padding='same', activation='relu')(x4)
  x4 = tf.keras.layers.BatchNormalization()(x4)

  combined_x = tf.keras.layers.Concatenate(axis=-1)([x1, x2, x3, x4])

  return tf.keras.layers.Add()([combined_x, x/0.5])

In [8]:
# input_x = tf.keras.Input(shape=(224, 224, 3))

# x_1  = vgg16.get_layer('input_1')(input_x)

# x_2  = vgg16.get_layer('block1_conv1')(x_1)
# x_3  = vgg16.get_layer('block1_conv2')(x_2)
# x_4  = vgg16.get_layer('block1_pool')(x_3)


# x_5  = vgg16.get_layer('block2_conv1')(x_4)
# x_6  = vgg16.get_layer('block2_conv2')(x_5)
# x_7  = vgg16.get_layer('block2_pool')(x_6)


# x_8  = vgg16.get_layer('block3_conv1')(x_7)
# x_9  = vgg16.get_layer('block3_conv2')(x_8)
# x_10 = vgg16.get_layer('block3_conv3')(x_9)
# x_11 = vgg16.get_layer('block3_pool')(x_10)


# x_12 = vgg16.get_layer('block4_conv1')(x_11)
# x_13 = vgg16.get_layer('block4_conv2')(x_12)
# x_14 = vgg16.get_layer('block4_conv3')(x_13)
# x_15 = vgg16.get_layer('block4_pool')(x_14)

# X1   = vgg16.get_layer('block4_conv3')(rfem(x_13))

# x_16 = vgg16.get_layer('block5_conv1')(x_15)
# x_17 = vgg16.get_layer('block5_conv2')(x_16)
# x_18 = vgg16.get_layer('block5_conv3')(x_17)
# x_19 = vgg16.get_layer('block5_pool')(x_18)

# X2   = vgg16.get_layer('block5_conv3')(rfem(X1))

# x_20 = tf.keras.layers.Conv2D(filters=1024, kernel_size=3, padding='same', activation='relu', name='block6_conv1')(x_19)
# x_21 = tf.keras.layers.MaxPool2D(pool_size=2, name='block6_pool')(x_20)

# X3   = tf.keras.layers.Conv2D(filters=1024, kernel_size=3, padding='same', activation='relu', name='block6_conv1')(rfem(X2))


# x_22 = tf.keras.layers.Conv2D(filters=512, kernel_size=3, strides=2, padding='same', activation='relu', name='block7_conv1')(x_21)
# x_23 = tf.keras.layers.MaxPool2D(pool_size=2, name='block7_pool')(x_22)

# X4   = tf.keras.layers.Conv2D(filters=512, kernel_size=3, strides=2, padding='same', activation='relu', name='block7_conv1')(rfem(X3))

# x_24 = tf.keras.layers.Flatten()(x_23)
# x_25 = tf.keras.layers.Dense(units=4096, activation='relu', name='dense_layer')(x_24)
# x_26 = tf.keras.layers.Dense(units=7, activation='softmax', name='final_layer')(x_25)


# X41  = ftb(X4)
# X42  = rfem(X41)

# X31  = ftb(X3, X41)
# X32  = rfem(X31)

# X21  = ftb(X2, X31)
# X22  = rfem(X21)

# X11  = ftb(X1, X21)
# X12  = rfem(X11)


# # PDM
# X4 + X42
# X3 + X32
# X2 + X22
# X1 + X12

In [9]:
# model = tf.keras.models.Model(inputs=input_x, outputs=[X])

In [10]:
# vgg16_short = tf.keras.models.Model(vgg16.input, vgg16.layers[-5].output)

# model_1 = tf.keras.Sequential([
#     vgg16_short, 
#     tf.keras.layers.Conv2D(filters=1024, kernel_size=(3, 3), padding='same', activation='relu', name='block6_conv1'),
#     tf.keras.layers.MaxPool2D(pool_size=2, name='block6_pool'),
#     tf.keras.layers.Conv2D(filters=512, kernel_size=(3, 3), strides=2, padding='same', activation='relu', name='block7_conv1'),
#     tf.keras.layers.MaxPool2D(pool_size=2, name='block7_pool'),
#     tf.keras.layers.Flatten(),
#     tf.keras.layers.Dense(units=4096, activation='relu', name='dense_layer'),
#     tf.keras.layers.Dense(units=7, activation='softmax', name='final_layer')
# ])
# model_1.summary()

In [11]:
# model_2 = tf.keras.models.Model(inputs=vgg16.input, outputs=[vgg16_short.get_layer('block4_conv2').output,
#                                                             vgg16_short.get_layer('block4_conv3').output,
#                                                             vgg16_short.get_layer('block5_conv3').output,
#                                                             model_1.get_layer('block6_conv1').output,
#                                                             model_1.get_layer('block7_conv1').output])

#Pytorch

In [84]:
class Encoder(torch.nn.Module):
  def __init__(self):
    super().__init__()
    model = torchvision.models.vgg16(weights='IMAGENET1K_V1')
    model.features = torch.nn.Sequential(*([model.features[i] for i in range(30)] + [model.features[i] for i in range(23, 30)]))
    return_layers = {'22': 'out_conv4_3', '29': 'out_conv5_3','34': 'out_conv7_2', '36': 'out_conv6_2'}
    self.backbone = IntermediateLayerGetter(model.features, return_layers=return_layers)
    device= torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
  def forward(self, images):
    return self.backbone(images)

In [85]:
x = torch.rand(5,3,512,512)
# y = model_with_multuple_layer(x)    

encoder = Encoder()
y = encoder(x)

In [86]:
y[0].keys(), y[1].shape

(odict_keys(['out_conv4_3', 'out_conv7_2', 'out_conv5_3', 'out_conv6_2']),
 torch.Size([5, 512, 16, 16]))

In [89]:
conv4_3 = y[0]['out_conv4_3']
conv5_3 = y[0]['out_conv5_3'][0]
conv6_2 = y[0]['out_conv6_2'][0]
conv7_2 = y[0]['out_conv7_2'][0]

print(conv4_3.shape, conv5_3.shape, conv6_2.shape, conv7_2.shape)

torch.Size([5, 512, 64, 64]) torch.Size([5, 512, 32, 32]) torch.Size([5, 512, 32, 32]) torch.Size([5, 512, 32, 32])


In [97]:
def ftb(current_layer, former_layer=None):
  x = torch.nn.Conv2d(current_layer.shape[1], 256, kernel_size=3, stride=1, padding='same')(current_layer)
  x = torch.nn.ReLU(inplace=True)(x)
  x = torch.nn.Conv2d(x.shape[1], 256, kernel_size=3, padding='same')(x)

  if former_layer is None:
    return x
  
  elif x.shape == former_layer.shape:
    return torch.add(x, former_layer)

  else:
    d = torch.nn.ConvTranspose2d(former_layer.shape[1], 256, kernel_size=4, stride=2, aadding='same')(former_layer)
    return torch.add(x, d)

ftb(conv4_3, conv5_3)


TypeError: ignored